In [ ]:
import os
os.chdir('modules')
from model import LogisticRegression, BernoulliNB, ComplementNB
from get_data import get_data
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
import time

X_dev, Y_dev = get_data("dev")

for size in [10, 100, 500, 2000]:
    tic = time.perf_counter()
    X_gpt_all, Y_gpt_all = get_data("gpt_" + str(size))
    X_gpt, Y_gpt = X_gpt_all[:-size], Y_gpt_all[:-size]
    X_base, Y_base = X_gpt_all[-size:], Y_gpt_all[-size:]
    model = LogisticRegression(max_iter=100)
    model.fit(X_base, Y_base)
    probs = [(list(probs), i) for i, probs in enumerate(model.predict_proba(X_gpt))]
    pruned_probs = []
    for (n_prob, p_prob), idx in probs:
        if n_prob > 0.8:
            if Y_gpt[idx] == 0:
                pruned_probs.append((n_prob,idx))
        elif n_prob < 0.2:
            if Y_gpt[idx] == 1:
                pruned_probs.append((n_prob, idx))
        else:
            pruned_probs.append((n_prob, idx))
    sorted_probs = sorted(pruned_probs, key=lambda x:x[0])
    final_probs = sorted_probs
    gpt_indices = [i[1] for i in final_probs]
    X_gpt_pruned = [X_gpt[i] for i in gpt_indices]
    Y_gpt_pruned = [Y_gpt[i] for i in gpt_indices]
    X_all = X_base + X_gpt_pruned
    Y_all = Y_base + Y_gpt_pruned
    LR = LogisticRegression(max_iter=100, ngram_range=(1, 1), min_df=5, max_df=1., verbose_vocab=True)
    CNB = ComplementNB(ngram_range=(1, 1), min_df=5, max_df=1., verbose_vocab=True)
    models = [LR,CNB]
    for model in models:
        model.fit(X_all, Y_all)
        acc = (model.predict(X_dev) == np.array(Y_dev)).mean()
        print(model['clf'],size,acc)
        toc = time.perf_counter()
        print(toc-tic)
        

Fitted vocab size: 909


/home/alanispani/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression() 10 0.7440917516218721
4.207272300000113
Fitted vocab size: 909
ComplementNB() 10 0.7454819277108434
6.657134099999894
Fitted vocab size: 2300


In [1]:
import os
#os.chdir('modules')
from model import LogisticRegression, BernoulliNB, ComplementNB
from get_data import get_data
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

for size in [10, 50, 100, 500, 2000]:
    X_base, Y_base = get_data("n_" + str(size), early_return=False)
    X_dev, Y_dev = get_data("dev")
    LR = LogisticRegression(max_iter=100)
    BNB = BernoulliNB()
    CNB = ComplementNB()
    models = [LR,BNB,CNB]
    for model in models:
        model.fit(X_base, Y_base)
        acc = (model.predict(X_dev) == np.array(Y_dev)).mean()
        print(model,acc)

ModuleNotFoundError: No module named 'model'

In [6]:
from sklearn.ensemble import StackingClassifier

for size in [100, 500, 2000]:
    X_gpt_all, Y_gpt_all = get_data("gpt_" + str(size))
    X_gpt, Y_gpt = X_gpt_all[:-size], Y_gpt_all[:-size]
    X_base, Y_base = X_gpt_all[-size:], Y_gpt_all[-size:]
    X_dev, Y_dev = get_data("dev")
    ps, scores = [], []
    model = LogisticRegression(max_iter=100)
    model.fit(X_base, Y_base)
    probs = [(list(probs), i) for i, probs in enumerate(model.predict_proba(X_gpt))]
    pruned_probs = []
    for (n_prob, p_prob), idx in probs:
        if n_prob > 0.8:
            if Y_gpt[idx] == 0:
                pruned_probs.append((n_prob,idx))
        elif n_prob < 0.2:
            if Y_gpt[idx] == 1:
                pruned_probs.append((n_prob, idx))
        else:
            pruned_probs.append((n_prob, idx))
    sorted_probs = sorted(pruned_probs, key=lambda x:x[0])
    #final_probs = sorted_probs[:size*25]+sorted_probs[-size*25:]
    final_probs = sorted_probs
    gpt_indices = [i[1] for i in final_probs]
    X_gpt_pruned = [X_gpt[i] for i in gpt_indices]
    Y_gpt_pruned = [Y_gpt[i] for i in gpt_indices]
    X_all = X_base + X_gpt_pruned
    Y_all = Y_base + Y_gpt_pruned
    estimators = [
         ('lr', LogisticRegression(max_iter=200)),
         ('bnb', BernoulliNB()),
        ('cnb', ComplementNB())
    ]
    clf = StackingClassifier(
         estimators=estimators, final_estimator=LogisticRegression()
    )

    clf.fit(X_all, Y_all)
    acc = (clf.predict(X_dev) == np.array(Y_dev)).mean()
    print(size,acc)


/home/alanispani/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


AttributeError: 'numpy.ndarray' object has no attribute 'lower'